In [98]:
from PIL import Image
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from dotenv import load_dotenv
from utils import load_img_path_from_dir, img_to_array
from data_transformation import data_transformation

In [ ]:
mopdel_1 = True
# alternativa de modelos : >>>> VGG16, restnet, google vision
def model_2_init():
    input_shape=(255,255,3)
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32,kernel_size=(5,5),input_shape=input_shape,padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(64,kernel_size=(5,5),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(4,4)))
    model.add(tf.keras.layers.Conv2D(128,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.Conv2D(160,kernel_size=(4,4),padding='same',activation='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(3,3)))
    model.add(tf.keras.layers.Conv2D(256,kernel_size=(3,3),padding='same',activation='relu'))
    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(64,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(30,activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(2,activation='softmax'))

    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=[tf.keras.metrics.Recall()])
    return model


In [ ]:
model = model_2_init()

In [ ]:
model.summary()

In [ ]:
dir_train = os.environ.get('TRAIN_PATH')
dir_test = os.environ.get('TEST_PATH')
dir_val = os.environ.get('VAL_PATH')

In [ ]:
dataset_train = image_dataset_from_directory(
  dir_train,
  labels = "inferred",
  label_mode = "categorical",
  seed=123,
  image_size=(255,255),
  batch_size=8)

In [ ]:
dataset_val = image_dataset_from_directory(
    dir_val,
    labels = "inferred",
    label_mode = "categorical",
    seed=123,
    image_size=(255,255),
    batch_size=8
)

In [ ]:
dataset_test = image_dataset_from_directory(
    dir_test,
    labels = 'inferred',
    label_mode = 'categorical',
    seed=123,
    image_size=(255,255),
    batch_size=8,
)

In [ ]:
# dataset_train = datagen.flow(dataset_train)

# Apply data augmentation and prefetch the data
# dataset_train = dataset_train.map(lambda x, y: (datagen.random_transform(x), y)).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
#es_recall = EarlyStoppingByRecall(monitor='val_recall', value=0.90, verbose=1)
es = EarlyStopping(patience=1,restore_best_weights=True)  # Early Stopping normal para la pérdida
model = model_2_init()
history = model.fit(dataset_train,validation_data=dataset_val,epochs=20,callbacks=[es])

In [ ]:
# load the images from tests/0 and predict
img_tests_0 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/0')
predictions_O = []
for img in img_tests_0:
    img = img_to_array(img)
    predictions_O.append(model.predict(img))

In [118]:
# load the images from tests/1 and predict
img_tests_1 = load_img_path_from_dir(os.environ.get('TEST_PATH')+ '/1')
predictions_1 = []
for img in img_tests_1:
    img = img_to_array(img)
    img_transformation = data_transformation(img, os.environ.get('TEST_PATH')+ '/t', 1)
    predictions_1.append(model.predict(img))

./Datasets/Dataset_test/1/IMG_1524.jpg
1/1 [==============================] - 0s 38ms/step
./Datasets/Dataset_test/1/IMG_1525.jpg
1/1 [==============================] - 0s 38ms/step
./Datasets/Dataset_test/1/IMG_1519.jpg
1/1 [==============================] - 0s 45ms/step
./Datasets/Dataset_test/1/IMG_1527.jpg
1/1 [==============================] - 0s 38ms/step
./Datasets/Dataset_test/1/IMG_1526.jpg
1/1 [==============================] - 0s 40ms/step
./Datasets/Dataset_test/1/IMG_1687.jpg
1/1 [==============================] - 0s 41ms/step
./Datasets/Dataset_test/1/IMG_1495.jpg
1/1 [==============================] - 0s 41ms/step
./Datasets/Dataset_test/1/IMG_1522.jpg
1/1 [==============================] - 0s 39ms/step
./Datasets/Dataset_test/1/IMG_1523.jpg
1/1 [==============================] - 0s 42ms/step
./Datasets/Dataset_test/1/IMG_1494.jpg
1/1 [==============================] - 0s 53ms/step
./Datasets/Dataset_test/1/IMG_1521.jpg
1/1 [==============================] - 0s 43ms/step

In [ ]:
# Obtener la categoria con más probabilidad
for res in predictions_O:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
for res in predictions_1:
    dog_id = np.argmax(res.max())
    print(dog_id)

In [ ]:
len(predictions_1)

In [ ]:
len(predictions_O)